# CWS_V1

In [11]:
# -*- coding:utf-8 -*-
from keras.layers import Dense, Embedding, LSTM, TimeDistributed, Input, Bidirectional
from keras.models import Model
from keras.utils import np_utils

import re
import numpy as np
import pandas as pd
word_size = 128
maxlen = 32
s = open('./../dataset/msr_train.txt').read().decode('gbk')
s = s.split('\r\n')

In [12]:
def clean(s): #整理一下数据，有些不规范的地方
    if u'“/s' not in s:
        return s.replace(u' ”/s', '')
    elif u'”/s' not in s:
        return s.replace(u'“/s ', '')
    elif u'‘/s' not in s:
        return s.replace(u' ’/s', '')
    elif u'’/s' not in s:
        return s.replace(u'‘/s ', '')
    else:
        return s

In [13]:
s = u''.join(map(clean, s))
s = re.split(u'[，。！？、]/[bems]', s)

data = [] #生成训练样本
label = []

In [14]:
def get_xy(s):
    s = re.findall('(.)/(.)', s)
    if s:
        s = np.array(s)
        return list(s[:,0]), list(s[:,1])

In [15]:
for i in s:
    x = get_xy(i)
    if x:
        data.append(x[0])
        label.append(x[1])

In [16]:
#转移概率，单纯用了等概率
zy = {'be':0.5, 
      'bm':0.5, 
      'eb':0.5, 
      'es':0.5, 
      'me':0.5, 
      'mm':0.5,
      'sb':0.5, 
      'ss':0.5
     }

zy = {i:np.log(zy[i]) for i in zy.keys()}

def viterbi(nodes):
    paths = {'b':nodes[0]['b'], 's':nodes[0]['s']}
    for l in range(1,len(nodes)):
        paths_ = paths.copy()
        paths = {}
        for i in nodes[l].keys():
            nows = {}
            for j in paths_.keys():
                if j[-1]+i in zy.keys():
                    nows[j+i]= paths_[j]+nodes[l][i]+zy[j[-1]+i]
            k = np.argmax(nows.values())
            paths[nows.keys()[k]] = nows.values()[k]
    return paths.keys()[np.argmax(paths.values())]

In [17]:
def simple_cut(s):
    if s:
        r = model.predict(np.array([list(chars[list(s)].fillna(0).astype(int))+[0]*(maxlen-len(s))]), verbose=False)[0][:len(s)]
        r = np.log(r)
        nodes = [dict(zip(['s','b','m','e'], i[:4])) for i in r]
        t = viterbi(nodes)
        words = []
        for i in range(len(s)):
            if t[i] in ['s', 'b']:
                words.append(s[i])
            else:
                words[-1] += s[i]
        return words
    else:
        return []

In [18]:
#转移概率，单纯用了等概率
zy = {'be':0.5, 
      'bm':0.5, 
      'eb':0.5, 
      'es':0.5, 
      'me':0.5, 
      'mm':0.5,
      'sb':0.5, 
      'ss':0.5
     }

zy = {i:np.log(zy[i]) for i in zy.keys()}

def viterbi(nodes):
    paths = {'b':nodes[0]['b'], 's':nodes[0]['s']}
    for l in range(1,len(nodes)):
        paths_ = paths.copy()
        paths = {}
        for i in nodes[l].keys():
            nows = {}
            for j in paths_.keys():
                if j[-1]+i in zy.keys():
                    nows[j+i]= paths_[j]+nodes[l][i]+zy[j[-1]+i]
            k = np.argmax(nows.values())
            paths[nows.keys()[k]] = nows.values()[k]
    return paths.keys()[np.argmax(paths.values())]

def simple_cut(s):
    if s:
        r = model.predict(np.array([list(chars[list(s)].fillna(0).astype(int))+[0]*(maxlen-len(s))]), verbose=False)[0][:len(s)]
        r = np.log(r)
        nodes = [dict(zip(['s','b','m','e'], i[:4])) for i in r]
        t = viterbi(nodes)
        words = []
        for i in range(len(s)):
            if t[i] in ['s', 'b']:
                words.append(s[i])
            else:
                words[-1] += s[i]
        return words
    else:
        return []

not_cuts = re.compile(u'([\da-zA-Z ]+)|[。，、？！\.\?,!]')
def cut_word(s):
    result = []
    j = 0
    for i in not_cuts.finditer(s):
        result.extend(simple_cut(s[j:i.start()]))
        result.append(s[i.start():i.end()])
        j = i.end()
    result.extend(simple_cut(s[j:]))
    return result

In [19]:
d = pd.DataFrame(index=range(len(data)))
d['data'] = data
d['label'] = label
d = d[d['data'].apply(len) <= maxlen]
d.index = range(len(d))
tag = pd.Series({'s':0, 'b':1, 'm':2, 'e':3, 'x':4})


chars = [] #统计所有字，跟每个字编号
for i in data:
    chars.extend(i)

chars = pd.Series(chars).value_counts()
chars[:] = range(1, len(chars)+1)

In [20]:
#生成适合模型输入的格式
d['x'] = d['data'].apply(lambda x: np.array(list(chars[x])+[0]*(maxlen-len(x))))
d['y'] = d['label'].apply(lambda x: np.array(map(lambda y:np_utils.to_categorical(y,5), tag[x].values.reshape((-1,1)))+[np.array([[0,0,0,0,1]])]*(maxlen-len(x))))

In [24]:
#设计模型
word_size = 128
maxlen = 32

sequence = Input(shape=(maxlen,), dtype='int32')
embedded = Embedding(len(chars)+1, word_size, input_length=maxlen, mask_zero=True)(sequence)
blstm = Bidirectional(LSTM(64, return_sequences=True,implementation=1), merge_mode='sum')(embedded)
output = TimeDistributed(Dense(5, activation='softmax'))(blstm)
model = Model(inputs=sequence, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 32, 128)           660864    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 32, 64)            98816     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 32, 5)             325       
Total params: 760,005
Trainable params: 760,005
Non-trainable params: 0
_________________________________________________________________


In [25]:
batch_size = 1024
history = model.fit(np.array(list(d['x'])), np.array(list(d['y'])).reshape((-1,maxlen,5)),
                    batch_size=batch_size,
                    verbose=2,
                    epochs=50)

Epoch 1/50
86s - loss: 0.7370 - acc: 0.7131
Epoch 2/50
87s - loss: 0.4079 - acc: 0.8532
Epoch 3/50
87s - loss: 0.3471 - acc: 0.8753
Epoch 4/50
87s - loss: 0.3118 - acc: 0.8885
Epoch 5/50
87s - loss: 0.2861 - acc: 0.8983
Epoch 6/50
87s - loss: 0.2651 - acc: 0.9062
Epoch 7/50
88s - loss: 0.2478 - acc: 0.9126
Epoch 8/50
88s - loss: 0.2324 - acc: 0.9182
Epoch 9/50
89s - loss: 0.2191 - acc: 0.9230
Epoch 10/50
89s - loss: 0.2072 - acc: 0.9274
Epoch 11/50
89s - loss: 0.1968 - acc: 0.9312
Epoch 12/50
89s - loss: 0.1874 - acc: 0.9345
Epoch 13/50
89s - loss: 0.1787 - acc: 0.9378
Epoch 14/50
89s - loss: 0.1708 - acc: 0.9407
Epoch 15/50
89s - loss: 0.1637 - acc: 0.9433
Epoch 16/50
89s - loss: 0.1570 - acc: 0.9459
Epoch 17/50
90s - loss: 0.1509 - acc: 0.9479
Epoch 18/50
90s - loss: 0.1450 - acc: 0.9502
Epoch 19/50
90s - loss: 0.1394 - acc: 0.9522
Epoch 20/50
90s - loss: 0.1346 - acc: 0.9540
Epoch 21/50
90s - loss: 0.1296 - acc: 0.9558
Epoch 22/50
90s - loss: 0.1251 - acc: 0.9574
Epoch 23/50
89s - l